# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import folium
import requests

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Define the base URL for Geoapify API
base_url = "https://api.geoapify.com/v1"

# Create a map centered at a specific location
m = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

# Add a marker for each city with the size based on humidity
for index, row in city_data_df.iterrows():
    params = {
        "lat": row["Lat"],
        "lon": row["Lon"],
        "zoom": 10,
        "apiKey": geoapify_key
    }

    response = requests.get(base_url, params=params).json()

    if response.get("map"):
        folium.CircleMarker(
            location=[row["Lat"], row["Lon"]],
            radius=row["Humidity"],  # Set the radius based on humidity
            popup=row["City"],
            color='blue',
            fill=True,
            fill_color='blue'
        ).add_to(m)

# Display the map
m

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit specific criteria
filtered_cities = city_data_df[city_data_df['Cloudiness'] <= 15]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
print(filtered_cities.head())

### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Create DataFrame hotel_df with selected columns from city_data_df
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lon', 'Humidity']].copy()

# Add an empty column for Hotel Name
hotel_df['Hotel Name'] = ''

# Display sample data
print(hotel_df.head())

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000  # Set the radius to 10,000 meters (10 km)
params = {
    "radius": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lon']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lon},{radius}"
    params["bias"] = f"point:{lat},{lon}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot with custom popup content
map_plot = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

for index, row in hotel_df.iterrows():
    popup_content = f"{row['City']}, {row['Country']}<br>Hotel: {row['Hotel Name']}"
    folium.Marker(
        location=[row['Lat'], row['Lon']],
        popup=popup_content
    ).add_to(map_plot)

# Display the map
map_plot